<a href="https://colab.research.google.com/github/NazninNaharSr/Dynamic-Vision/blob/main/DAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Load the CSV file and preprocess the data
file_path = '/content/RangeData15_Oct_2024_1353_22_864.csv'  # Update with your new dataset
gesture_data = pd.read_csv(file_path)

# Selecting relevant features for gesture classification
selected_features = [
    'rng__no_of_targets', 'rng__zone_id', 'peak_rate_kcps_per_spad',
    'range_sigma_mm', 'median_range_mm', 'min_range_delta_mm',
    'max_range_delta_mm', 'target_reflectance_est_pc', 'target_status'
]

# Filtering the dataset to keep only the selected features
gesture_data_filtered = gesture_data[selected_features]

# Normalizing the features using StandardScaler
scaler = StandardScaler()
gesture_data_normalized = scaler.fit_transform(gesture_data_filtered)

# Updated gesture labels based on provided image
gesture_labels = np.random.choice(
    ['thumb-up', 'thumb-down', 'palm-left', 'palm-right', 'palm-up', 'palm-down', 'palm-forward', 'palm-backward'],
    size=len(gesture_data_normalized)
)

# Splitting the dataset into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    gesture_data_normalized, gesture_labels, test_size=0.2, random_state=42
)

# Convert data into PyTorch tensors and DataLoader
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor([np.where(np.unique(gesture_labels) == label)[0][0] for label in y_train], dtype=torch.long)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor([np.where(np.unique(gesture_labels) == label)[0][0] for label in y_test], dtype=torch.long)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the Transformer-based model for gesture classification
class GestureDATModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=128, nhead=4, num_layers=2, dim_feedforward=512, dropout=0.1):
        super(GestureDATModel, self).__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)  # Add sequence dimension
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)  # Global average pooling
        out = self.fc_out(x)
        return out

# Parameters for the model
input_dim = X_train.shape[1]
num_classes = len(np.unique(gesture_labels))

# Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GestureDATModel(input_dim=input_dim, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Evaluating the model on the test set
model.eval()
correct = 0
total = 0
predicted_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predicted_labels.extend(predicted.cpu().numpy())

print(f"Test Accuracy: {100 * correct / total:.2f}%")

# Mapping predictions back to gesture labels
gesture_label_mapping = {i: label for i, label in enumerate(np.unique(gesture_labels))}
final_predictions = [gesture_label_mapping[pred] for pred in predicted_labels]

# Load the new CSV file and predict the gesture based on the trained model
new_file_path = '/content/RangeData15_Oct_2024_1353_22_864.csv'  # Update with the new dataset
data_to_predict = pd.read_csv(new_file_path)

# Filtering the dataset to keep only the selected features
data_to_predict_filtered = data_to_predict[selected_features]

# Normalizing the features using the same scaler used for training
data_to_predict_normalized = scaler.transform(data_to_predict_filtered)

# Convert data into PyTorch tensors
X_new_tensor = torch.tensor(data_to_predict_normalized, dtype=torch.float32).to(device)

# Predict the gesture
model.eval()
with torch.no_grad():
    outputs = model(X_new_tensor)
    _, predicted = torch.max(outputs, 1)
    predicted_gesture = gesture_label_mapping[predicted[0].item()]

# Display the predicted gesture
print(f"Predicted Gesture: {predicted_gesture}")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch [1/10], Loss: 2.1459
Epoch [2/10], Loss: 2.1065
Epoch [3/10], Loss: 2.0931
Epoch [4/10], Loss: 2.1033
Epoch [5/10], Loss: 2.0875
Epoch [6/10], Loss: 2.0839
Epoch [7/10], Loss: 2.0852
Epoch [8/10], Loss: 2.0891
Epoch [9/10], Loss: 2.0782
Epoch [10/10], Loss: 2.0739
Test Accuracy: 13.31%
Predicted Gesture: thumb-down
